In [247]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import classification_report, confusion_matrix,mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier,XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from statsmodels.distributions.empirical_distribution import ECDF
import datetime

In [128]:
import pickle
with open('./강남구데이터/grid_coordi', 'rb') as f:
    a = pickle.load(f)
with open('./강남구데이터/grid_to_zero_coordi', 'rb') as f:
    b = pickle.load(f) 

In [129]:
grid_gangnam1=df(a).T
grid_gangnam2=df(b).T
grid_gangnam=pd.concat([grid_gangnam1,grid_gangnam2])
grid_gangnam=gpd.GeoDataFrame(grid_gangnam,geometry=gpd.points_from_xy(grid_gangnam[1],grid_gangnam[0]))

# 1.주어진 데이터 - 시간, 위도, 경도

In [101]:
example

,Time,위도,경도
1984523,2020-08-03 20:12,37.526538,127.028703
1984524,2020-08-03 20:14,37.523858,127.022980
1984525,2020-08-03 20:16,37.499020,127.062182
1984526,2020-08-03 20:18,37.499020,127.062182
1984527,2020-08-03 20:18,37.514379,127.052918
...,...,...,...
1984617,2020-08-03 23:20,37.517263,127.036615
1984618,2020-08-03 23:25,37.507023,127.044023
1984619,2020-08-03 23:26,37.507023,127.044023
1984620,2020-08-03 23:30,37.507307,127.024679


# 2.feature로 변환

In [229]:
data=gpd.GeoDataFrame(example,geometry=gpd.points_from_xy(example.경도,example.위도))

In [230]:
def extract_feature(data):
    def grid_match(data_i):
        grid_match=np.load('./grid/grid_match',allow_pickle=True)
        lat_min = 37.455
        lat_max = 37.535
        long_min = 127.01
        long_max = 127.125

        lat_bin = np.arange(lat_min, lat_max, 0.005)
        lat_bin = np.append(lat_bin, lat_max)

        long_bin = np.arange(long_min,long_max, 0.005)
        long_bin = np.append(long_bin, long_max)
        for i in range(139):
            a = grid_match[i][0]
            b = grid_match[i][1]
            if (data_i.위도 >= lat_bin[a]) & (data_i.위도 < lat_bin[a+1]):
                if (data_i.경도 >= long_bin[b]) & (data_i.경도 < long_bin[b+1]):
                    return i
            
    data['time']=data['Time'].apply(lambda i : datetime.datetime.strptime(i,'%Y-%m-%d %H:%M').hour)
    data['day']=data['Time'].apply(lambda i : datetime.datetime.strptime(i,'%Y-%m-%d %H:%M').day)
    data['grid_index']=-1
    for i in data.index:
        data.loc[i,'grid_index']=grid_match(data.loc[i,:])
    

In [231]:
extract_feature(data)

# 3.변환된 데이터의 결과
## 이 때 grid_index가 NaN인 경우 단속되지 않는다고 보면 된다.

In [233]:
data

,Time,위도,경도,geometry,time,day,grid_index
1984523,2020-08-03 20:12,37.526538,127.028703,POINT (127.02870 37.52654),20,3,NaN
1984524,2020-08-03 20:14,37.523858,127.022980,POINT (127.02298 37.52386),20,3,NaN
1984525,2020-08-03 20:16,37.499020,127.062182,POINT (127.06218 37.49902),20,3,80.0
1984526,2020-08-03 20:18,37.499020,127.062182,POINT (127.06218 37.49902),20,3,80.0
1984527,2020-08-03 20:18,37.514379,127.052918,POINT (127.05292 37.51438),20,3,117.0
...,...,...,...,...,...,...,...
1984617,2020-08-03 23:20,37.517263,127.036615,POINT (127.03661 37.51726),23,3,NaN
1984618,2020-08-03 23:25,37.507023,127.044023,POINT (127.04402 37.50702),23,3,104.0
1984619,2020-08-03 23:26,37.507023,127.044023,POINT (127.04402 37.50702),23,3,104.0
1984620,2020-08-03 23:30,37.507307,127.024679,POINT (127.02468 37.50731),23,3,NaN


# 이 중 하나의 경우에 대해서 예측을 한다고 하면

In [234]:
data_i=data.loc[data.index[2],:]

In [280]:
data_i

Time                                   2020-08-03 20:16
위도                                               37.499
경도                                              127.062
geometry      POINT (127.062182304456 37.4990204274664)
time                                                 20
day                                                   3
grid_index                                           80
Name: 1984525, dtype: object

In [235]:
from joblib import dump, load
enc = load('enc.joblib') 

In [236]:
processed_data=[data_i[['grid_index','time','day']].tolist()]
processed_data=enc.transform(processed_data)

# 4. 해당 데이터가 단속될 것인지, 아닌 것인지를 분류
## 단속되지 않을 확률이 70%이상이면 단속되지 않는다고 함
## 단속되지 않을 확률이 70% 미만일 경우 단속될 수 있다고 생각해서 다음 모델로 넘어감

In [237]:
# model load
xgb_1=XGBClassifier()
xgb_1.load_model('model1.model')

In [238]:
# predict
# 단속되지 않을 확률
wasted_prob=xgb_1.predict_proba(processed_data)[:,0]
print(wasted_prob)

[0.35591137]


# 단속되지 않을 확률이 70% 미만일 경우

# 5. 해당 데이터가 어느 정도의 확률도 단속될 지를 예측

## 5.1 해당 데이터가 어느 그리드들과 인접한 지를 계산 (5개 정도)

In [239]:
distance=grid_gangnam.geometry.distance(data_i.geometry)

In [242]:
distance

0      0.054100
1      0.051039
2      0.048303
3      0.053930
4      0.050365
         ...   
164    0.041596
165    0.045370
166    0.057818
167    0.062218
168    0.052406
Length: 169, dtype: float64

## 가까운 k개와의 distance와 grid 를 추출

In [243]:
top_k=np.argsort(distance)[:5]
top_k_distance=distance[np.argsort(distance)[:5]]

In [281]:
top_k

0    80
1    95
2    79
3    81
4    94
dtype: int64

In [282]:
top_k_distance

80    0.001553
95    0.003494
79    0.004923
81    0.005531
94    0.005834
dtype: float64

## 이를 토대로 다시 feature를 생성
## 하지만 grid_to_zero coordinate의 경우는 제외시킴 

In [244]:
real_top_k=[ i for i in top_k if i not in b.keys() ]
real_top_k_distance = x[real_top_k]

not_real_top_k=[ i for i in top_k if i in b.keys() ]
not_real_top_k_distance = x[not_real_top_k]

In [284]:
new_data=[[i,data_i.time,data_i.day] for i in real_top_k]

In [286]:
df(new_data,columns=['grid_index','time','data'])

,grid_index,time,data
0,80,20,3
1,95,20,3
2,79,20,3
3,81,20,3
4,94,20,3


## real top k로만 feature를 만들어 내고, 분석을 진행함

In [253]:
new_data=enc.transform(new_data)

In [254]:
# model load
xgb_2=XGBRegressor()
xgb_2.load_model('model2.model')

In [255]:
# 예측
nums=xgb_2.predict(new_data)

In [257]:
from statsmodels.distributions.empirical_distribution import ECDF
y_train=np.load('./empirical_num.npy')
ecdf=ECDF(y_train)

In [287]:
nums

array([2.3752651, 2.15125  , 2.4657044, 2.3361168, 2.3784719],
      dtype=float32)

In [258]:
# 확률로 변환
probs=ecdf(nums)

In [259]:
probs

array([0.73050784, 0.73050784, 0.73050784, 0.73050784, 0.73050784])

## 최종 산출물

In [277]:
answer = (probs * softmax(real_top_k_distance)).sum()

In [269]:
def softmax(x):
    s=(np.exp(x)).sum()
    return np.exp(x)/s

In [279]:
print(answer)

0.7305078372049296


In [278]:
answer

0.7305078372049296